# Elastic Stackの構築手順：インベントリの設定
----
Ansibleで利用するインベントリ情報を ファイルに出力します。

具体的には、[01_01_Outline.ipynbの設定の出力](01_01_Outline.ipynb#設定の出力)で生成されたインベントリのテンプレート(hosts_template)に、  
01_02_Accommodation_XXX.ipynb で生成したサーバのIPアドレス一覧を適用し、  
インベントリファイルを作成します。

## ホストリストの設定

01_02_Accommodation_XXX.ipynb の「Inventory作成用のインスタンスリストを作成」で生成したホストのリストの文字列をコピーし、以下のセルに貼り付けて実行してください。

In [1]:
#host_list = {'host1':('name', 'public ip', 'private ip')}
host_list = 

ホスト定義の値を確認しておきます。

In [2]:
host_list

{'host1': (u'i-0bf412a529d12xxxx', u'34.213.xx.xx', u'10.30.xxx.xxx')}

これ以降のNotebookで使用するためのAPIエンドポイントのパブリックIPアドレスをファイルに記録しておきます。

In [3]:
with open('rc.py', 'w') as f:
    f.write(u'''es_host = "{}"
USER = 'centos'
KEYPATH = '~/.ssh/ansible_id_rsa'
'''.format(host_list['host1'][1]))
!cat rc.py

es_host = "34.213.xx.xx"
USER = 'centos'
KEYPATH = '~/.ssh/ansible_id_rsa'


## 設定
アカウントや通信先などを設定します。

各サーバにAnsibleコマンドを発行するため、  
Ansibleを実行可能なユーザアカウントとそのSSHキーを次のセルで設定してください。

In [4]:
#elasticsearchをインストールするサーバーでansibleコマンドを実行するユーザー
USER='centos'

#公開鍵認証を行う場合の秘密鍵のパス
#KEYPATH='/notebooks/etc/key/id_rsa'
KEYPATH = '~/.ssh/ansible_id_rsa'

## 既存のインベントリのバックアップ
出力前に、既存の内容をバックアップ用ディレクトリに退避します。

バックアップ用ディレクトリを生成します。

In [5]:
from datetime import datetime as dt
import os
bak_dir = './bak_' + dt.now().strftime('%Y%m%d')
if(not os.path.isdir(bak_dir)) : os.mkdir(bak_dir)
print(bak_dir)

./bak_20170730


ファイルを生成したディレクトリにコピーします。

In [6]:
if os.path.exists('hosts'):
    !cp ./hosts {bak_dir}/hosts

次のセルを実行すると、上のセルで記録したIPリストをインベントリのテンプレート(hosts_template)と合成し、  
新しいhostsファイルを生成します。  
生成したhostsの内容は画面にも出力されます。

In [7]:
%run scripts/get-replace-file.py
replace_file('hosts_template', 'hosts', host_list)
!cat ./hosts

[all]
i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx

[master-nodes]
i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx

[data-nodes]
i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx

[ingest-nodes]
i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx

[logstash-server]
i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx


## 内容の確認・修正
生成したインベントリファイルの内容を確認、修正したり、前回のバージョンと比較したい場合は以下から実施できます。

インベントリに出力しているグループの意味と、そのグループに応じたこの後の処理は次の通りです。  
各グループに重複して同じサーバアドレスを記入することは、特に問題ありません。

|グループ|グループに入れるべきサーバ|設定によるこのあとの実際の処理の違い|
|---|---|---|
|[master-nodes]|master-eligible node<br>（Master Nodeの候補）にしたいサーバ|elasticsearch.ymlの設定内容を変更します。<br>グループ内のサーバか否かで、<br>**"node.master"**<br>をtrue/falseに切り替えます。<br>また、**"discovery.zen.ping.unicast.hosts"**<br>にグループ内のホスト名またはIPアドレスを配列で指定します。|
|[data-nodes]|Data Nodeにしたいサーバ|elasticsearch.ymlの設定内容を変更します。<br>グループ内のサーバか否かで、<br>**"node.data"**<br>をtrue/falseに切り替えます。|
|[ingest-nodes]|Ingest Nodeにしたいサーバ|elasticsearch.ymlの設定内容を変更します。<br>グループ内のサーバか否かで、<br>**"node.ingest"**<br>をtrue/falseに切り替えます。|
|[logstash-server]|Logstash用にしたいサーバ|グループ内のサーバか否かで、<br>Logstashをインストールするかを切り替えます。|


出力されたhosts のファイルを修正したい場合は、次のリンクから編集してください。

- **[hosts](../edit/hosts)**

変更結果を既存の内容（バックアップディレクトリに退避しているもの）と比較するには  
次のセルを実行してください。

In [8]:
!diff -ur {bak_dir}/hosts hosts || exit 0

--- ./bak_20170730/hosts	2017-07-30 12:50:21.453174247 +0000
+++ hosts	2017-07-30 12:50:22.142384586 +0000
@@ -1,28 +1,14 @@
 [all]
-i-070e599a2f340f55a ansible_ssh_host=35.164.172.32
-i-0f83d4a2a48ff74cf ansible_ssh_host=52.40.22.166
-i-02400d21bcac8ecdb ansible_ssh_host=52.26.116.204
-i-0171aabd7aed539fb ansible_ssh_host=52.34.182.253
-i-083741ca08986ec92 ansible_ssh_host=34.213.150.135
+i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx
 
 [master-nodes]
-i-070e599a2f340f55a ansible_ssh_host=35.164.172.32
-i-0f83d4a2a48ff74cf ansible_ssh_host=52.40.22.166
-i-02400d21bcac8ecdb ansible_ssh_host=52.26.116.204
+i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx
 
 [data-nodes]
-i-070e599a2f340f55a ansible_ssh_host=35.164.172.32
-i-0f83d4a2a48ff74cf ansible_ssh_host=52.40.22.166
-i-02400d21bcac8ecdb ansible_ssh_host=52.26.116.204
-i-0171aabd7aed539fb ansible_ssh_host=52.34.182.253
-i-083741ca08986ec92 ansible_ssh_host=34.213.150.135
+i-0bf412a529d12xxxx ansible_ssh_host=34.213.xx.xx
 
 [in

## インベントリのテスト
作成したインベントリにある各ホストに、Ansibleから各種の操作が可能か確認します。


### ホストに到達可能か？
インベントリの各ホストに、Ansibleからアクセス可能か確認します。

次のコマンドを実行し、全ホストにPINGを発行してください。

In [9]:
!ansible all -m ping -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


各サーバから、次のようにSUCCESSという応答が返ってくれば問題なく実行できています。

### 適切なユーザで認証されるか？

Ansibleからホストへの認証は、一般ユーザとして認証されるか確認します。

次のコマンドを実行してください。

In [10]:
!ansible -a 'whoami' all -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS | rc=0 >>
centos



各サーバからSUCCESSという結果と、[設定](#設定)の章で設定したユーザ名が次のように応答されていれば、問題なく認証できています。

### 対象ホストでsudo可能か？
インベントリの各ホストに、Ansibleからsudo可能か確認します。


次のコマンドを実行し、全ホストにsudoしてください。

In [11]:
!ansible --become --become-user root -a 'whoami' all -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS | rc=0 >>
root



各サーバからSUCCESSという結果と、rootというユーザ名が次のように応答されていれば、問題なくsudoできています。

### Ansibleによる操作ログが記録されているか？
Ansibleで各ホストを操作した結果を、過去の分も含めて参照できるか確認します。

ログのディレクトリまたはファイルを指定してください。

In [12]:
LOG_PATH = '/var/log/*'

次のコマンドを実行し、ansibleの実行ログが出現するか確認してください。

In [13]:
!ansible -b -m shell -a 'grep ansible- {LOG_PATH} | tail -n 2' all -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS | rc=0 >>
/var/log/secure:Jul 30 12:50:52 ip-10-30-117-160 sudo:  centos : TTY=pts/0 ; PWD=/home/centos ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-egfkmnfeaorsjnsluqpgfnkxvcxylgpf; /usr/bin/python /home/centos/.ansible/tmp/ansible-tmp-1501419048.65-137031746094503/command.py; rm -rf "/home/centos/.ansible/tmp/ansible-tmp-1501419048.65-137031746094503/" > /dev/null 2>&1
/var/log/secure:Jul 30 12:50:58 ip-10-30-117-160 sudo:  centos : TTY=pts/0 ; PWD=/home/centos ; USER=root ; COMMAND=/bin/sh -c echo BECOME-SUCCESS-ynjmgfnxtmivahhwrpggrfmkghpwhlms; /usr/bin/python /home/centos/.ansible/tmp/ansible-tmp-1501419054.08-52730567935613/command.py; rm -rf "/home/centos/.ansible/tmp/ansible-tmp-1501419054.08-52730567935613/" > /dev/null 2>&1grep: /var/log/anaconda: Is a directory
grep: /var/log/audit: Is a directory
grep: /var/log/chrony: Is a directory
grep: /var/log/ppp: Is a directory
grep: /var/log/rhsm: Is a directory
grep: /var/log/tuned: Is a direct

※長期運用にあたっては、実行ログがディスクを圧迫しないようrotateされるかを確認してください。  

## /etc/hostsの更新

各仮想マシンの `/etc/hosts` に、インスタンスIDを名前としてPrivate IPにアクセスできるよう設定します。

In [14]:
for name, public_ip, private_ip in host_list.values():
    !ansible -b -m lineinfile -a 'path=/etc/hosts line="{private_ip} {name}"' all -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "msg": "line added"
}


変更後の`/etc/hosts` の内容を確認しておく。

In [15]:
!ansible -a 'cat /etc/hosts' all -i ./hosts -u $USER --private-key=$KEYPATH

i-0bf412a529d12xxxx | SUCCESS | rc=0 >>
127.0.0.1   localhost localhost.localdomain localhost4 localhost4.localdomain4
::1         localhost localhost.localdomain localhost6 localhost6.localdomain6

10.30.xxx.xxx i-0bf412a529d12xxxx

